In [1]:
import json
from scipy.stats import spearmanr, pearsonr

In [2]:
s = open("results/q-future/q-align-koniq-spaq-v0/combined-kadid_ref.json").read()
s = '[' + s.replace('}{', '},{') + ']'
refs = json.loads(s)

In [3]:
s = open("results/q-future/q-align-koniq-spaq-v0/combined-kadid.json").read()
s = '[' + s.replace('}{', '},{') + ']'
res = json.loads(s)

In [4]:
def rescale(x):
    import numpy as np
    x = np.array(x)
    x = (x - x.mean()) / x.std()
    return 1 + np.exp(x)

def wa5(logits):
    import numpy as np
    logprobs = np.array([logits["excellent"], logits["good"], logits["fair"], logits["poor"], logits["bad"]])
    probs = np.exp(logprobs) / np.sum(np.exp(logprobs))
    return np.inner(probs, np.array([1,0.75,0.5,0.25,0.]))

In [5]:
prs_refs = {}
for ri in refs:
    prs_refs[ri["id"].split(".")[0]] = wa5(ri["logits"])
prs = rescale([wa5(ri["logits"]) - prs_refs[ri["id"].split("_")[0]] for ri in res])
gts = [float(ri["id"].split(">")[-1]) for ri in res]
print(spearmanr(prs,gts)[0],pearsonr(prs,gts)[0])

(0.7365033771016285, 0.7251340946166804)